In [28]:
#import packages
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import json

In [29]:
game_data = pd.read_csv('data/games.csv')

#remove 999usd game
# i = game_data[(game_data.AppID == 1200520)].index
# game_data.drop(i[0], inplace=True)

game_data.shape

(70210, 39)

### Remove unnecessary columns (not relevant to data analysis) <br>
URLs/Contact info: website, support url, support email, metacritic url <br>
Images/videos: screenshots, movies, header image <br>
irrelevant/unusable data: user score(mostly NaN), score rank (mostly NULL), no. of achivements <br>
<br>
remove utilites: not games

In [30]:
removed_columns = ["Header image", "Website", "Support url", "Support email", "Metacritic url", "User score", "Score rank", "Achievements", "Screenshots", "Movies"]
game_data.drop(columns=removed_columns, inplace=True)
game_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70210 entries, 0 to 70209
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       70210 non-null  int64  
 1   Name                        70209 non-null  object 
 2   Release date                70210 non-null  object 
 3   Estimated owners            70210 non-null  object 
 4   Peak CCU                    70210 non-null  int64  
 5   Required age                70210 non-null  int64  
 6   Price                       70210 non-null  float64
 7   DLC count                   70210 non-null  int64  
 8   About the game              67909 non-null  object 
 9   Supported languages         70210 non-null  object 
 10  Full audio languages        70210 non-null  object 
 11  Reviews                     9121 non-null   object 
 12  Windows                     70210 non-null  bool   
 13  Mac                         702

In [31]:
#remove all utilities
game_data = game_data.loc[~game_data['Genres'].str.contains(r"utilities", case=False, na=False)]
game_data = game_data.loc[~game_data['Tags'].str.contains(r"utilities", case=False, na=False)]
game_data = game_data.loc[~game_data['Categories'].str.contains(r"utilities", case=False, na=False)]

Cleaning Categories

In [32]:
cat_price = pd.DataFrame(game_data[["AppID", "Name", "Price", "Categories"]])

In [33]:
#generate table of categories
a = cat_price.head(1)
json_table = []
cat_price_json = cat_price.to_json(orient='records')
cat_price_json = json.loads(cat_price_json)

for j in cat_price_json:
    test = {}
    cats = j["Categories"]
    test["AppID"] = j["AppID"]
    test["Name"] = j["Name"]
    test["Price"] = j["Price"]
    
    if cats != None:
        cats = cats.split(",")
        for i in cats:
            test[i] = "T"
    
    json_table.append(test)


print(len(json_table))

69358


In [34]:
cat_price_table = pd.DataFrame(json_table)
print(cat_price_table.shape)
print(cat_price_table.columns.tolist())
cat_price_table.fillna(value="F", inplace=True)

cat_price_table.head()

(69358, 43)
['AppID', 'Name', 'Price', 'Single-player', 'Multi-player', 'Steam Achievements', 'Partial Controller Support', 'Full controller support', 'Steam Leaderboards', 'Remote Play on Phone', 'Remote Play on Tablet', 'Remote Play on TV', 'MMO', 'PvP', 'Online PvP', 'Co-op', 'Online Co-op', 'In-App Purchases', 'Steam Cloud', 'Steam Trading Cards', 'Shared/Split Screen', 'Cross-Platform Multiplayer', 'Remote Play Together', 'Stats', 'Shared/Split Screen PvP', 'Captions available', 'Steam Workshop', 'Includes level editor', 'LAN PvP', 'LAN Co-op', 'Shared/Split Screen Co-op', 'Steam Turn Notifications', 'VR Support', 'SteamVR Collectibles', 'Valve Anti-Cheat enabled', 'Includes Source SDK', 'Commentary available', 'Mods (require HL2)', 'Mods', 'Tracked Motion Controller Support', 'Tracked Controller Support', 'VR Only', 'VR Supported']


,AppID,Name,Price,Single-player,Multi-player,Steam Achievements,Partial Controller Support,Full controller support,Steam Leaderboards,Remote Play on Phone,...,SteamVR Collectibles,Valve Anti-Cheat enabled,Includes Source SDK,Commentary available,Mods (require HL2),Mods,Tracked Motion Controller Support,Tracked Controller Support,VR Only,VR Supported
0,20200,Galactic Bowling,19.99,T,T,T,T,F,F,F,...,F,F,F,F,F,F,F,F,F,F
1,655370,Train Bandit,0.99,T,F,T,F,T,T,T,...,F,F,F,F,F,F,F,F,F,F
2,1732930,Jolt Project,4.99,T,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,F
3,1355720,Henosis™,5.99,T,F,F,F,T,F,F,...,F,F,F,F,F,F,F,F,F,F
4,1139950,Two Weeks in Painland,0.00,T,F,T,F,F,F,F,...,F,F,F,F,F,F,F,F,F,F


Cleaning Genres

In [35]:
genre_price = pd.DataFrame(game_data[["AppID", "Name", "Price", "Genres"]])

In [36]:
#generate table of genres
a = genre_price.head(1)
json_table = []
genre_price_json = genre_price.to_json(orient='records')
genre_price_json = json.loads(genre_price_json)

for j in genre_price_json:
    test = {}
    gens = j["Genres"]
    test["AppID"] = j["AppID"]
    test["Name"] = j["Name"]
    test["Price"] = j["Price"]
    
    if gens != None:
        gens = gens.split(",")
        for i in gens:
            test[i] = "T"
    
    json_table.append(test)

print(len(json_table))

69358


In [37]:
gen_price_table = pd.DataFrame(json_table)
print(gen_price_table.shape)
print(gen_price_table.columns.tolist())
gen_price_table.fillna(value="F", inplace=True)

gen_price_table.head()

(69358, 30)
['AppID', 'Name', 'Price', 'Casual', 'Indie', 'Sports', 'Action', 'Adventure', 'Strategy', 'Free to Play', 'Massively Multiplayer', 'RPG', 'Simulation', 'Early Access', 'Racing', 'Education', 'Sexual Content', 'Nudity', 'Violent', 'Gore', 'Design & Illustration', 'Game Development', 'Software Training', 'Animation & Modeling', 'Video Production', 'Photo Editing', 'Web Publishing', 'Audio Production', 'Accounting', 'Movie']


,AppID,Name,Price,Casual,Indie,Sports,Action,Adventure,Strategy,Free to Play,...,Design & Illustration,Game Development,Software Training,Animation & Modeling,Video Production,Photo Editing,Web Publishing,Audio Production,Accounting,Movie
0,20200,Galactic Bowling,19.99,T,T,T,F,F,F,F,...,F,F,F,F,F,F,F,F,F,F
1,655370,Train Bandit,0.99,F,T,F,T,F,F,F,...,F,F,F,F,F,F,F,F,F,F
2,1732930,Jolt Project,4.99,F,T,F,T,T,T,F,...,F,F,F,F,F,F,F,F,F,F
3,1355720,Henosis™,5.99,T,T,F,F,T,F,F,...,F,F,F,F,F,F,F,F,F,F
4,1139950,Two Weeks in Painland,0.00,F,T,F,F,T,F,F,...,F,F,F,F,F,F,F,F,F,F


In [38]:
# gen_price_table.to_csv("gen.csv", sep=',', encoding='utf-8')

In [39]:
clean_gen = gen_price_table.loc[((gen_price_table['Casual'] == 'T') | (gen_price_table['Indie']  == "T") | (gen_price_table['Sports']  == "T") | (gen_price_table['Action']  == "T") | (gen_price_table['Adventure']  == "T") | (gen_price_table['Strategy']  == "T") | (gen_price_table['Free to Play']  == "T") | (gen_price_table['Massively Multiplayer']  == "T") | (gen_price_table['RPG']  == "T") | (gen_price_table['Simulation']  == "T") | (gen_price_table['Racing']  == "T"))]
# clean_gen.to_csv("gen2.csv", sep=',', encoding='utf-8')

In [40]:
clean_ids = clean_gen["AppID"]
clean_ids.to_list()

[20200,
 655370,
 1732930,
 1355720,
 1139950,
 1469160,
 1659180,
 1968760,
 1178150,
 320150,
 1026420,
 485000,
 1620060,
 825930,
 1454010,
 22670,
 1330820,
 346560,
 1097880,
 1362670,
 290870,
 575760,
 434030,
 810740,
 2073470,
 589250,
 677880,
 1294780,
 1342850,
 1177420,
 793990,
 955490,
 552350,
 42500,
 1195610,
 1095380,
 1158910,
 1194940,
 262150,
 1192900,
 866150,
 231330,
 1056520,
 897820,
 658970,
 1777550,
 552520,
 718350,
 433450,
 1709450,
 1431470,
 1794920,
 1959480,
 763550,
 1445370,
 464230,
 1293830,
 1509640,
 1135380,
 1944730,
 1144900,
 306350,
 1684530,
 1557480,
 1128520,
 12140,
 1647430,
 1845050,
 1163340,
 1608620,
 541570,
 1119070,
 1292520,
 1467330,
 1624600,
 870000,
 1621490,
 768060,
 1746130,
 1281400,
 372890,
 1023420,
 1141880,
 1022840,
 652160,
 968580,
 1131550,
 1729790,
 368080,
 572510,
 1225870,
 1232510,
 1066890,
 817370,
 1662060,
 457140,
 1774810,
 1886290,
 1514360,
 1374280,
 863490,
 396640,
 726020,
 424060,
 199524

In [41]:
clean_table = game_data[game_data['AppID'].isin(clean_ids)]
clean_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66524 entries, 0 to 70209
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       66524 non-null  int64  
 1   Name                        66524 non-null  object 
 2   Release date                66524 non-null  object 
 3   Estimated owners            66524 non-null  object 
 4   Peak CCU                    66524 non-null  int64  
 5   Required age                66524 non-null  int64  
 6   Price                       66524 non-null  float64
 7   DLC count                   66524 non-null  int64  
 8   About the game              66447 non-null  object 
 9   Supported languages         66524 non-null  object 
 10  Full audio languages        66524 non-null  object 
 11  Reviews                     8988 non-null   object 
 12  Windows                     66524 non-null  bool   
 13  Mac                         665

Limit dataset to games with more than x reviews

In [42]:
rev_table = pd.DataFrame(game_data)
rev_table['Rate_Ratio'] = rev_table['Positive']/ (rev_table['Negative'] + rev_table['Positive'])
rev_table['Reviews'] = rev_table['Negative'] + rev_table['Positive']

rev_table.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Rate_Ratio
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",0.352941
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",0.913793
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,NaN
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",1.000000
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",0.862069


In [43]:
rev_table.dropna(axis=0, subset=["Rate_Ratio"], inplace=True)
small_review_price = rev_table[rev_table['Reviews'] >= 75]

In [44]:
small_review_price.describe()

,AppID,Peak CCU,Required age,Price,DLC count,Reviews,Metacritic score,Positive,Negative,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Rate_Ratio
count,1.827200e+04,18272.000000,18272.000000,18272.000000,18272.000000,1.827200e+04,18272.000000,1.827200e+04,18272.000000,1.827200e+04,18272.000000,18272.000000,18272.000000,18272.000000,18272.000000
mean,7.446457e+05,521.668290,0.884906,10.800267,1.574704,5.005314e+03,14.170808,4.304051e+03,701.262916,3.466527e+03,403.173927,43.692535,342.906907,46.834665,0.780851
std,4.685952e+05,11388.998649,3.732790,11.500150,18.351147,6.030447e+04,29.292549,5.227075e+04,9837.493988,3.829291e+04,1850.172373,394.816966,2357.927374,428.674573,0.157122
min,1.000000e+01,0.000000,0.000000,0.000000,0.000000,7.500000e+01,0.000000,2.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.025316
25%,3.667900e+05,0.000000,0.000000,1.990000,0.000000,1.450000e+02,0.000000,1.077500e+02,28.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.695949
50%,6.392200e+05,2.000000,0.000000,7.990000,0.000000,3.400000e+02,0.000000,2.580000e+02,67.000000,1.910000e+02,81.000000,0.000000,74.000000,0.000000,0.816999
75%,1.082508e+06,14.000000,0.000000,14.990000,1.000000,1.233000e+03,0.000000,9.800000e+02,214.250000,7.350000e+02,290.000000,0.000000,273.000000,0.000000,0.902162
max,2.258590e+06,872138.000000,21.000000,269.990000,1555.000000,6.531097e+06,97.000000,5.764420e+06,895978.000000,3.441592e+06,90351.000000,19159.000000,136629.000000,19159.000000,1.000000


Check for unneeded data

In [54]:
comp = "Categories"
genre_price = pd.DataFrame(small_review_price[["AppID", "Name", "Price", comp]])
genre = pd.DataFrame(game_data[comp])
u_gen = genre_price[comp].unique()
print(genre_price[comp].describe())
print(len(u_gen))
gen_list = []
for i in u_gen:
    try:
        arr = i.split(",")
    except:
        print(i)
    for j in arr:
        if j not in gen_list:
            gen_list.append(j)

print(gen_list)
print(len(gen_list))

count             18212
unique             3445
top       Single-player
freq               2120
Name: Categories, dtype: object
3446
nan
['Single-player', 'Multi-player', 'MMO', 'PvP', 'Online PvP', 'Co-op', 'Online Co-op', 'In-App Purchases', 'Steam Achievements', 'Full controller support', 'Steam Trading Cards', 'Partial Controller Support', 'Steam Cloud', 'Remote Play on Tablet', 'Shared/Split Screen', 'Cross-Platform Multiplayer', 'Remote Play Together', 'Stats', 'Steam Leaderboards', 'Remote Play on Phone', 'Shared/Split Screen PvP', 'Captions available', 'Steam Workshop', 'Includes level editor', 'LAN Co-op', 'LAN PvP', 'Remote Play on TV', 'Shared/Split Screen Co-op', 'Steam Turn Notifications', 'VR Support', 'SteamVR Collectibles', 'Valve Anti-Cheat enabled', 'Commentary available', 'Includes Source SDK', 'Mods (require HL2)', 'Mods', 'Tracked Controller Support', 'VR Only']
38


In [46]:
#create csv for cleaned data
small_review_price.to_csv("data/clean_data.csv", sep=',', encoding='utf-8')